In [1]:
import xarray as xr
import rioxarray as rio
import os
from glob import glob

In [5]:
os.chdir(r'J:\cmip6\data\thetao_mdmean')
years = list(range(2051, 2100))
# ['historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
for depth in ['surface', 'mesopelagic', 'bathypelagic', 'abyssopelagic']:
    os.mkdir("%s" % depth)
    for scen in ['ssp126', 'ssp245', 'ssp370', 'ssp585']:
        os.mkdir("%s/%s" % (depth, scen))
#         for yr in years:
#             os.mkdir("%s/%s/%s" % (depth, scen, yr))

In [ ]:
for scen in ['ssp245', 'ssp370', 'ssp585']:
    for yr in years:
        os.chdir(r'J:\cmip6\data\thetao_add\%s\%s' % (scen, yr))
        files = sorted(glob('*.nc'))
        for f in files:
            ds = xr.open_dataset(f, decode_times=False)
            dirname = f.split('_')[2]
            os.mkdir('J:/cmip6/data/thetao_depth/surface/%s/%s/%s' % (scen, yr, dirname))
            os.mkdir('J:/cmip6/data/thetao_depth/mesopelagic/%s/%s/%s' % (scen, yr, dirname))
            os.mkdir('J:/cmip6/data/thetao_depth/bathypelagic/%s/%s/%s' % (scen, yr, dirname))
            os.mkdir('J:/cmip6/data/thetao_depth/abyssopelagic/%s/%s/%s' % (scen, yr, dirname))
                
            try:
                # ds为二维元组
                for h,data in ds.groupby('lev'):
                    data = data.thetao 
                    h = int(round(h, 0)) # 深度

                    ## 分类保存
                    if h < 200:
                        out_name = os.path.join('J:\\cmip6\\data\\thetao_depth\\surface\\%s\\%s\\%s' % (scen, yr, dirname), 
                                                '%s.tif' % h)
                    elif h < 1000:
                        out_name = os.path.join('J:\\cmip6\\data\\thetao_depth\\mesopelagic\\%s\\%s\\%s' % (scen, yr, dirname), 
                                                '%s.tif' % h)
                    elif h < 4000:
                        out_name = os.path.join('J:\\cmip6\\data\\thetao_depth\\bathypelagic\\%s\\%s\\%s' % (scen, yr, dirname), 
                                                '%s.tif' % h)
                    else:
                        out_name = os.path.join('J:\\cmip6\\data\\thetao_depth\\abyssopelagic\\%s\\%s\\%s' % (scen, yr, dirname), 
                                                '%s.tif' % h)

                    outdata = data.transpose('time', 'lat', 'lon')
                    outdata = outdata.rio.set_spatial_dims(x_dim='lon',y_dim='lat')
                    outdata = outdata.rio.set_crs("epsg:4326")
                    outdata.rio.to_raster(out_name) 

            except:
                pass
            
        print("year: %s --- finished" % yr)
            
    print(" ## == %s --- finished == ##" % scen)